![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

Exercise material of the MSc-level course **Advanced Soil Mechanics**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/fakultaet3/gt/soilmechanics


# Von Spannungen, Verzerrungen und Verschiebungen zu Setzungen -- direkte Setzungsberechnungen

Die folgenden Überlegungen dienen der Darstellung grundlegender mechanischer Zusammenhänge, bevor die praktischen Setzungsberechnungsverfahren eingeführt werden.

In [1]:
#HIDDEN
import numpy as np #numerical methods
import matplotlib.pyplot as plt #plotting
import seaborn as sns
import sympy as sp
sp.init_printing(use_latex='mathjax') #makes sympy output look nice

#Some plot settings
import plot_functions.plot_settings
%run plot_functions/settlements_plots.ipynb

## Wiederholung: Lineare Elastizität in 3D

Die Effektivspannungsänderungen, die durch den Lasteintrag hervorgerufen werden, gehen mit Verformungen einher. Die Verbindung von Spannungs- und Verzerrungsänderungen erfolgt über konstitutive Modelle, also Material- oder Stoffmodelle.

Der linear elastische Festkörper wird im dreidimensionalen Raum beschrieben durch das generalisierte Hooke'sche Gesetz:

\begin{align}
\left(
    \begin{array}{c}
    \epsilon_{xx}
    \\
    \epsilon_{yy}
    \\
    \epsilon_{zz}
    \\
    \epsilon_{xy}
    \\
    \epsilon_{yz}
    \\
    \epsilon_{xz}
    \end{array}
\right)
=
\frac{1}{E}
\left(
    \begin{array}{cccccc}
    1 & -\nu & -\nu & 0 & 0 & 0
    \\
    -\nu & 1 & -\nu & 0 & 0 & 0
    \\
    -\nu & -\nu & 1 & 0 & 0 & 0
    \\
    0 & 0 & 0 & 1 + \nu & 0 & 0
    \\
    0 & 0 & 0 & 0 & 1 + \nu & 0
    \\
    0 & 0 & 0 & 0 & 0 & 1 + \nu
    \end{array}
\right)
\left(
    \begin{array}{c}
    \Delta \sigma_{xx}'
    \\
    \Delta \sigma_{yy}'
    \\
    \Delta \sigma_{zz}'
    \\
    \Delta \sigma_{xy}'
    \\
    \Delta \sigma_{yz}'
    \\
    \Delta \sigma_{xz}'
    \end{array}
\right)
\end{align}

bzw. nach den Spannungen aufgelöst

\begin{align}
\left(
    \begin{array}{c}
    \Delta \sigma_{xx}'
    \\
    \Delta \sigma_{yy}'
    \\
    \Delta \sigma_{zz}'
    \\
    \Delta \sigma_{xy}'
    \\
    \Delta \sigma_{yz}'
    \\
    \Delta \sigma_{xz}'
    \end{array}
\right)
=
\frac{E}{(1+\nu) (1-2\nu)}
\left(
    \begin{array}{cccccc}
    1 -\nu & \nu & \nu & 0 & 0 & 0
    \\
    \nu & 1 - \nu & \nu & 0 & 0 & 0
    \\
    \nu & \nu & 1 - \nu & 0 & 0 & 0
    \\
    0 & 0 & 0 & 1 - 2\nu & 0 & 0
    \\
    0 & 0 & 0 & 0 & 1 - 2\nu & 0
    \\
    0 & 0 & 0 & 0 & 0 & 1 - 2\nu
    \end{array}
\right)
\left(
    \begin{array}{c}
    \epsilon_{xx}
    \\
    \epsilon_{yy}
    \\
    \epsilon_{zz}
    \\
    \epsilon_{xy}
    \\
    \epsilon_{yz}
    \\
    \epsilon_{xz}
    \end{array}
\right)
\end{align}

Für die Verzerrungen wurde dabei auf die Ausweisung der Inkremente $\Delta$ verzichtet, da implizit angenommen wird, dass der Primärspannungszustand als unvervormter Ausgangszustand betrachtet wird.

## Verzerrungen im Halbraum

Uns interessieren nun vornehlich die Vertikalverschiebungen des Bodens an der Geländeoberkannte. Diese ergeben sich aus den sich über die Tiefe akkumulierenden Vertikaldehnungen:

$$
    s(x,y) = u_z(x,y)|_{z=0} = \int \limits_{z=0}^{z=\infty} \epsilon_{zz} \text{d}z
$$

Für die Vertikaldehnung finden wir die Beziehung

$$
    \epsilon_{zz} = \frac{1}{E} \left[ \Delta \sigma'_{zz} - \nu (\Delta \sigma'_{xx} + \Delta \sigma'_{yy})  \right],
$$

mit der Vertikalverformungen bei gegebenen Normalspannungsverteilungen berechnet werden können.

Im Sonderfall behinderter Seitendehnung, der oft unter ausgedehnten Lasten angenommen wird, ergeben sich die Horizontalspannungsinkremente zu

$$
    \Delta \sigma'_{xx} = \Delta \sigma'_{yy} = \frac{E \nu }{(1+\nu) (1-2\nu)} \epsilon_{zz}
$$

Eingesetzt in die obige Beziehung für die Vertikaldehnung ergibt sich dann

$$
    \epsilon_{zz} = \frac{\Delta \sigma'_{zz}}{E_\text{s}} \quad \text{mit} \quad  E_\text{s} = \frac{E(1-\nu)}{(1+\nu) (1-2\nu)}
$$

mit dem Steifemodul $E_\text{s}$ (Steifigkeit bei behinderter Seitendehnung). Im Folgenden wird implizit $\sigma'_{ij} = \sigma_{ij}$ angenommen.

## Isotroper elastischer Halbraum unter einer vertikalen Punktlast

Wir rufen uns die analytische Lösung nach Boussinesq in Erinnerung.

Die kartesischen Koordinaten $x,y,z$ ($z$ vertikal) haben ihren Ursprung am Angriffspunkt der Punktlast. Zusätzlich werden die horizontal radialen Abstände $r$ sowie der räumliche Abstant $R$ wie folgt definiert:

\begin{align}
    r &= \sqrt{x^2 + y^2}
    \\
    R &= \sqrt{r^2 + z^2}
\end{align}

Nach der analytischen Lösung von Boussinesq ergeben sich die folgenden Gleichungen für die drei Normalspannungsverläufe:

\begin{align}
    \Delta \sigma_{xx} &= \frac{3F_z}{2\pi R^2} \left[ \frac{x^2 z}{R^3} - \frac{1-2\nu}{3} \left( \frac{(x^2 - y^2) R}{r^2 (R + z)} + \frac{y^2 z}{R r^2}\right) \right]
    \\
    \Delta \sigma_{yy} &= \frac{3F_z}{2\pi R^2} \left[ \frac{y^2 z}{R^3} - \frac{1-2\nu}{3} \left( \frac{(y^2 - x^2) R}{r^2 (R + z)} + \frac{x^2 z}{R r^2}\right) \right]
    \\
    \Delta \sigma_{zz} &= \frac{3F_z}{2\pi R^2} \frac{z^3}{R^3}
\end{align}

Die Summe der beiden horizontalen Normalspannungen ergibt sich zu:

$$
    \Delta \sigma_{xx} + \Delta \sigma_{yy} = \frac{3F_z}{2\pi R^2} \left[ \frac{r^2 z}{R^3} - \frac{1-2\nu}{3} \frac{z}{R} \right]
$$

\begin{align}
    \epsilon_{zz} &= \frac{1}{E} \left[ \Delta \sigma'_{zz} - \nu (\Delta \sigma'_{xx} + \Delta \sigma'_{yy})  \right]
    \\
    &= \frac{3F_z}{2\pi R^2 E} \left[ \frac{z^3}{R^3} - \nu \left( \frac{r^2 z}{R^3} - \frac{1-2\nu}{3} \frac{z}{R} \right) \right]
\end{align}

Wir berechnen nun das Integral über $\epsilon_{zz}$ von $z=\infty$ bis zu einer Tiefe $\bar{z}$, also:

$$
    u_z(r,\bar{z}) = \frac{3F_z}{2\pi E} \int \limits_{\bar{z}}^{\infty} \frac{1}{R^2} \left[ \frac{z^3}{R^3} - \nu \left( \frac{r^2 z}{R^3} - \frac{1-2\nu}{3} \frac{z}{R} \right) \right] \text{d}z
$$

Dazu definieren wir in \texttt{sympy} den integranden \text{eps} (ohne konstanten Vorfaktor) und den Radius R:

In [2]:
r, z, nu, eps = sp.symbols('r z nu epsilon')
R = sp.sqrt(r**2 + z**2)
R

   _________
  ╱  2    2 
╲╱  r  + z  

In [3]:
eps = 1/R**2 * ((z/R)**3 - nu * (r**2*z/R**3 - (1-2*nu)/3 * z/R))
eps

    ⎛                 ⎛1   2⋅ν⎞ ⎞               
    ⎜     2         z⋅⎜─ - ───⎟ ⎟         3     
    ⎜    r ⋅z         ⎝3    3 ⎠ ⎟        z      
- ν⋅⎜──────────── - ────────────⎟ + ────────────
    ⎜         3/2      _________⎟            3/2
    ⎜⎛ 2    2⎞        ╱  2    2 ⎟   ⎛ 2    2⎞   
    ⎝⎝r  + z ⎠      ╲╱  r  + z  ⎠   ⎝r  + z ⎠   
────────────────────────────────────────────────
                     2    2                     
                    r  + z                      

In [4]:
sp.integrate(eps,(z,z,sp.oo)).simplify()

     2  2      2  2      2      2      2
- 2⋅ν ⋅r  - 2⋅ν ⋅z  + ν⋅z  + 2⋅r  + 3⋅z 
────────────────────────────────────────
                        3/2             
               ⎛ 2    2⎞                
             3⋅⎝r  + z ⎠                

Geschickte Umformung durch Ausklammern und Verwendung der dritten binomischen Formel liefert:

\begin{align}
    u_z(r,z) &= \frac{3F_z}{2\pi E} \frac{1}{3R} \left[ 2(1-\nu^2) + \frac{z^2}{R^2} ( 1+\nu) \right]
    \\
    &= \frac{F_z}{2\pi E} \frac{1+\nu}{R} \left[ 2(1-\nu) + \frac{z^2}{R^2} \right]
\end{align}

Der Schubmodul ist mit dem E-Modul und der Querkontraktionszahl über

$$
    G = \frac{E}{2(1+\nu)}
$$

verknüpft, so dass unser Ergebnis letztlich lautet

$$
    u_z(r,z) = \frac{F_z}{4\pi G R} \left[ 2(1-\nu) + \frac{z^2}{R^2} \right]
$$

Oberflächensetzungen ergeben sich direkt zu

$$
    s(r) = u_z(r,z=0) = \frac{F_z(1-\nu)}{2\pi G r}
$$

Man erkennt in dieser Lösung wiederum die Singularität am Punkt des Lasteintrages.

Zum Vergleich berechnen wir nun noch die Vertikalverschiebung bei Verwendung der Annahme einachsiger Verformungen (bei der Punktlast eine fragwürdige Annahme).

$$
    \epsilon_{zz} = \frac{\Delta \sigma'_{zz}}{E_\text{s}} = \frac{3F_z}{2\pi E_\text{s} R^2} \frac{z^3}{R^3}
$$

Damit ergibt sich

$$
    \bar{u}_z(r,\bar{z}) = \frac{3F_z}{2\pi E_\text{s}} \int \limits_{\bar{z}}^{\infty} \frac{1}{R^2} \left[ \frac{z^3}{R^3} \right] \text{d}z
$$


In [5]:
eps = 1/R**2 * ((z/R)**3)
eps

      3     
     z      
────────────
         5/2
⎛ 2    2⎞   
⎝r  + z ⎠   

In [6]:
sp.integrate(eps,(z,z,sp.oo)).simplify()

    2       
 2⋅r     2  
 ──── + z   
  3         
────────────
         3/2
⎛ 2    2⎞   
⎝r  + z ⎠   

Umformung ergibt für die Vertikalverschiebung

$$
    \bar{u}(r,z) = \frac{F_z}{2\pi E_\text{s} R} \left[ 2 + \frac{z^2}{R^2} \right]
$$

und für die Setzung

$$
    \bar{s}(r) = \bar{u}(r,z=0) = \frac{F_z}{\pi E_\text{s} r}
$$

*Aufgabe*: Wie unterscheiden sich die Lösungen $s(r)$ und $\bar{s}(r)$ bei $\nu = 0$? Wie erklärt sich das?

Wir stellen nun beide Lösungen graphisch dar.

In [7]:
#HIDDEN
interactive_point_settlement()

interactive(children=(BoundedFloatText(value=100.0, description='$E$ / MPa', max=1000.0, min=0.001), BoundedFl…

Es treten größere Abweichungen auf, da unter der Punktlast nicht nur Kompressionssetzungen, sondern auch Verdrängungssetzungen auftreten.

## Isotroper elastischer Halbraum unter konstanter Rechtecklast

Aufgrund des Superpositionsprinzips kann die obige Lösung u.a. so integriert werden, dass die Eckpunktsetzungen einer $a \times b$ großen Rechtecklast der Intensität $q$ ermittelt werden können. Führt man die Integration des Vertikaldehnungsfeldes in den veränderten Grenzen 

$$
    s = \int \limits_{z=0}^{z=t} \epsilon_{zz} \text{d}z
$$

aus, also bis in eine endliche Tiefe $t$ durch, so entspricht das einer in der Tiefe $t$ anstehenden undeformierbaren Baugrundschicht. Die Eckpunktsetzung ergibt sich zu

$$
    s = \frac{q (1-\nu)^2}{\pi E} \left[
        a \ln \frac{\left( b + \sqrt{a^2 + b^2} \right) \sqrt{a^2 + t^2}}{a(b+R)} + 
        b \ln \frac{\left( a + \sqrt{a^2 + b^2} \right) \sqrt{b^2 + t^2}}{b(a+R)}
    \right] + 
    \frac{q (1-\nu-2\nu^2)}{2 \pi E} t \arctan \left( \frac{ab}{tR} \right)
$$

Den Einfluss der Integrationsgrenztiefe $t$ demonstriert die folgende Abbildung.

In [8]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interactive_rect_settlement()

interactive(children=(BoundedFloatText(value=100.0, description='$E$ / MPa', max=1000.0, min=0.001), BoundedFl…

Oberflächennah anstehende sehr steife Baugrundschichten verringern die auftretenden Setzungen also sehr stark. Aufgrund der Spannungsausbreitung im Untergrund tragen tiefere Schichten immer weniger zum Setzungsbetrag bei. Es lässt sich also die Existenz einer Grenztiefe für die Setzungsberechnung erwarten. Dazu das nächste Mal mehr.